# A Survival Guide to Group and Pivot
Two advanced operations that are extremely useful when working with data tables are .group() and .pivot(), but both are tricky to understand at first. The best way to learn both is by example. In this notebook we will use examples that are discussed in [Chapter 8](https://inferentialthinking.com/chapters/08/3/Cross-Classifying_by_More_than_One_Variable.html) of your textbook, *Inferential Thinking*.

In [ ]:
# Import the usual modules
import numpy as np
from datascience import *
%matplotlib inline

The table `cones` records the flavor, color, and price of six ice cream cones.

In [ ]:
cones = Table().with_columns(
    'Flavor', make_array('strawberry', 'chocolate', 'chocolate', 'strawberry', 'chocolate', 'bubblegum'),
    'Color', make_array('pink', 'light brown', 'dark brown', 'pink', 'dark brown', 'pink'),
    'Price', make_array(3.55, 4.75, 5.25, 5.25, 5.25, 4.75)
)
cones

## Grouping Example 1: Counting occurances based on one category
Suppose you wanted to know how many many rows are the color pink. Yes, for this small table you could easily count them, but what if the table had millions of rows?

One way would be to create a table with only the rows of a particular color using `.where`, then count the rows in that table w `.num_rows`

In [ ]:
num_pink = cones.where('Color', are.equal_to('pink')).num_rows
print(f"There are {num_pink} rows with pink ice cream.")

That works, but it would sure be tedious to repeat the process for every color. This is where grouping comes into play.

In [ ]:
cones.group('Color')

By default, .group() returns a new table with a row for each unique entry of the selected column and the count how many times that entry appeared in the original table. However, while counting is the default, other functions can be used to collect together the rows of each group.

Let's try this again, only this time we will group by flavor.

In [ ]:
cones.group('Flavor')

## Grouping Example 2: Counting occurances based on more than one category
To classify the cones by both flavor and color, we will pass a list of labels as an argument to group. The resulting table has one row for every unique combination of values that appear together in the grouped columns.

In [ ]:
cones.group(['Flavor', 'Color'])

## Grouping Example 3: Collecting the row with a function other than count.
When you group the rows, you are collecting all of the rows that meet a particular condition into one row per unique value in the column you use to group. Count() is not the one choice for the collection function.

In the example, we will group on flavor, but we will has for the total cost by flavor.

In [ ]:
cones.group('Flavor', sum)

As you can see, the result is the sum of prices for all of the rows by flavor. Verify this by comparing with the original table. Notice that name of the function your use is added to the label of the column created. Notice also although a `Color sum` column is created, it is empty because `sum` has no meaning when applied to strings.

Why did we not get this extra column when we used `count` in Example 2? Because the count is the same for every column.

In [ ]:
cones

You can use any function in the grouping, including a function you write yourself as long as it makes sense to apply the function when combining the rows in each group. 

Suppose we wanted to know the most expensive cone for each flavor.

In [ ]:
cones.group('Flavor', max)

Notice that name of the function your use is added to the label of the column created. Notice also that this time we have a `Color max` column. It turns out that `max` has a meaning when applied to strings.

In [ ]:
max('the', 'string', 'starting', 'with', 'the', 'letter', 'furthest', 'down', 'in', 'the', 'alphabet', 'is', 'the', 'max')

# Pivoting Example 1

In [ ]:
# Our original table for comparison
cones

In [ ]:
cones.pivot('Flavor', 'Color')

What just happened? Starting with `cones` table, we created a new table where the first label, `flavor`, became the columns and the second label, `Color` became the rows. The values of the table contains the counts: `tbl.pivot(col_label, row_label)`.

In the original `cones` table there were two chocolate cones that were dark brown, so in the pivot table the value is two. There were no dark brown bubble gum cones (ugh!), so the value is zero.

Just as with grouping, you can collect the values with functions other than `count`.

In [ ]:
cones.pivot('Flavor', 'Color', values='Price', collect=sum)

Compare with using `group`

In [ ]:
cones.group(['Flavor', 'Color'], sum)

Quoting from the textbook: "The advantage of pivot is that it places grouped values into adjacent columns, so that they can be combined and compared."

## Pivoting Example 2
To drive home the power of pivot tables, we will follow another example given in the text book. The focus will be on education and income for Californians in 2014.

In [ ]:
full_table = Table().read_table('../data/educ_inc.csv')

# Focus on adults in 2014
ca_2014 = full_table.where('Year', are.equal_to('1/1/14 0:00')).where('Age', are.not_equal_to('00 to 17'))
ca_2014

Focus on education and income.

In [ ]:
# Create a subset of the original table with select()
educ_inc = ca_2014.select('Educational Attainment', 'Personal Income', 'Population Count')
educ_inc

Use group to see how many Californians appear in each educational attainment category.

In [ ]:
education = educ_inc.select('Educational Attainment', 'Population Count')
educ_totals = education.group('Educational Attainment', sum)
educ_totals

In [ ]:
# Create a function that turns numbers in an array into percents rounded to two decimal places.
def percents(array_x):
    return np.round( (array_x / sum(array_x)) * 100, 2)

In [ ]:
# Always test your function
x = make_array(.100001, .15632, .2514)
percents(x)

Add a percent column

In [ ]:
educ_distribution = educ_totals.with_column(
    'Population Percent', percents(educ_totals.column(1))
)
educ_distribution

Now we `pivot` to look at income vs. education.

In [ ]:
totals = educ_inc.pivot('Educational Attainment', 'Personal Income', values='Population Count', collect=sum)
totals

**If you understand this next code cell, then you are really starting to undertand tables. If you don't, get someone to explain it to you.**

In [ ]:
distributions = totals.select('Personal Income').with_columns(
    "Bachelor's degree or higher", percents(totals.column(1)),
    'College, less than 4-yr degree', percents(totals.column(2)),
    'High school or equivalent', percents(totals.column(3)),
    'No high school diploma', percents(totals.column(4))   
    )

distributions

In [ ]:
distributions.select(0, 1, 4).barh(0)

**A not-so-subtle argument for higher education.**